In [1]:
def prep_data(data):
    from sklearn import preprocessing
    # Cabin more useful as leter and number - Some have multiple cabins, just take the first one
    data = data.drop('id',axis=1)
    data = data.drop('name',axis=1) # saw that this was being used in some trees, but i dont think it makes sense to use, probably causing overfitting
    data = data.drop('ticket',axis=1) # saw that this was being used in some trees, but i dont think it makes sense to use, probably causing overfitting
    data['cabin'] = data['cabin'].fillna('')
    data['cabin_letter'] = data['cabin'].str.slice(0,1)
    data['cabin_number'] = pd.to_numeric( data['cabin'].str.slice(1,3) ) # did bit of a naughty change on csv for this
    data = data.drop('cabin',axis=1)
    le = preprocessing.LabelEncoder()
    for column_name in data.columns:
        if data[column_name].dtype == object:
            print("Converting %s from %s" % (column_name, data[column_name].dtype))
            data[column_name] = le.fit_transform(data[column_name].fillna(''))
        else:
            data[column_name] = data[column_name].fillna(0)
    y = data.pop('survived')
    X = data
    
    feature_names = data.columns
    number_of_features = len(data.columns)
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    return feature_names, number_of_features, X_train, X_test, y_train, y_test
    


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from os import system
import numpy as np
import pandas as pd

C:\Users\oliver\Anaconda3\envs\Titanic_Environment\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
data = pd.read_csv('train.csv')
data.dtypes
feature_names, number_of_features, X_train, X_test, y_train, y_test = prep_data(data)


Converting embarked from object
Converting sex from object
Converting cabin_letter from object


In [4]:
#Single Tree
max_depths = [1,2,3,4,5,6,7,8,9,10]
for i in max_depths:
    dt_clf = DecisionTreeClassifier(max_depth=i)
    dt_clf.fit(X_train, y_train)
    dt_score = dt_clf.score(X_test,y_test)
    print("Single Decision Tree Score with Max Depth %i is %f" % (i , dt_score))
    

Single Decision Tree Score with Max Depth 1 is 0.759825
Single Decision Tree Score with Max Depth 2 is 0.759825
Single Decision Tree Score with Max Depth 3 is 0.772926
Single Decision Tree Score with Max Depth 4 is 0.764192
Single Decision Tree Score with Max Depth 5 is 0.777293
Single Decision Tree Score with Max Depth 6 is 0.768559
Single Decision Tree Score with Max Depth 7 is 0.781659
Single Decision Tree Score with Max Depth 8 is 0.781659
Single Decision Tree Score with Max Depth 9 is 0.781659
Single Decision Tree Score with Max Depth 10 is 0.764192


In [5]:
#Single Tree with HPT

decision_tree_clf_random = DecisionTreeClassifier()
#Parameters for Tuning
criterion         = ['gini','entropy']
max_depth         = [1,2,3,4,5,6,7,8,9,10,15,20,30,40,50,75,100]
min_samples_split = list(range(2,250,10))
max_features      = list(range(1,number_of_features)) + ['auto','sqrt','log2']


decision_tree_random_cv = {
    'criterion'        : criterion,
    'max_depth'        : max_depth,
    'min_samples_split': min_samples_split,
    'max_features'     : max_features
}

random_decision_tree = RandomizedSearchCV(
    estimator = decision_tree_clf_random,
    param_distributions = decision_tree_random_cv,
    n_iter = 500,
    cv = 5,
    verbose=1)
random_decision_tree.fit(X_train, y_train)

random_decision_tree_score = random_decision_tree.score(X_test,y_test)
print("score was")
print(random_decision_tree_score)
random_decision_tree_best_params = random_decision_tree.best_params_
print("best parameters")
print(random_decision_tree_best_params)


Fitting 5 folds for each of 500 candidates, totalling 2500 fits
score was
0.74235807860262
best parameters
{'min_samples_split': 32, 'max_features': 'auto', 'max_depth': 7, 'criterion': 'gini'}


[Parallel(n_jobs=1)]: Done 2500 out of 2500 | elapsed:    6.9s finished


In [6]:
#Single Tree with HPT

decision_tree_clf_grid = DecisionTreeClassifier()
#Parameters for Tuning
criterion         = ['gini','entropy']
max_depth         = [3,4,5,6,7,8,9,10]
min_samples_split = list(range(70,90,1))
max_features      = list(range(5,number_of_features)) + ['auto','sqrt','log2']


decision_tree_grid_cv = {
    'criterion'        : criterion,
    'max_depth'        : max_depth,
    'min_samples_split': min_samples_split,
    'max_features'     : max_features
}

grid_decision_tree = GridSearchCV(
    estimator = decision_tree_clf_grid,
    param_grid = decision_tree_grid_cv,
    cv = 5,
    verbose=1)
grid_decision_tree.fit(X_train, y_train)
grid_decision_tree_score       = grid_decision_tree.score(X_test,y_test)
print("score was")
print(grid_decision_tree_score)
grid_decision_tree_best_params = grid_decision_tree.best_params_ 
print("best parameters")
print(grid_decision_tree_best_params)


Fitting 5 folds for each of 2240 candidates, totalling 11200 fits
score was
0.7816593886462883
best parameters
{'criterion': 'gini', 'max_depth': 7, 'max_features': 5, 'min_samples_split': 72}


[Parallel(n_jobs=1)]: Done 11200 out of 11200 | elapsed:   31.3s finished


In [7]:
out_file_name = "titanic_tree_best_decision_tree.dot"
out_png_file_name = "titanic_tree_best_decision_tree.png"
export_graphviz(
    grid_decision_tree.best_estimator_,
    out_file=out_file_name,
    feature_names=feature_names,
    rounded=True,
    filled=True
    )


#Converts dot file to PNG
cmd_convert = 'dot -Tpng %s -o %s' % (out_file_name,out_png_file_name)
#system(cmd_convert) # Why doesnt this work!
#!dot -Tpng out_file_name -o out_png_file_name # Or this
print(cmd_convert)  # Print to run manually as above doesnt work

dot -Tpng titanic_tree_best_decision_tree.dot -o titanic_tree_best_decision_tree.png


In [8]:
#RandomForest No HPT
random_forest_clf = RandomForestClassifier()
random_forest_clf.fit(X_train,y_train)

random_forest_score = random_forest_clf.score(X_test,y_test)
print("Score for Random Forest classsifier with no tuning is")
print(random_forest_score)

Score for Random Forest classsifier with no tuning is
0.7729257641921398


In [9]:
#RamdomForest with Random HPT

random_forest_clf_random = RandomForestClassifier() # Parameter Tuning here
#Parameters for Tuning
criterion         = ['gini','entropy']
n_estimators      = list(range(100,1000,10))
max_depth         = [1,2,3,4,5,6,7,8,9,10,15,20]
min_samples_split = list(range(2,250,10))
max_features      = list(range(5,number_of_features)) + ['auto','sqrt','log2']
bootstrap         = ['True','False']


random_forest_random_cv = {
    'criterion'        : criterion,
    'n_estimators'     : n_estimators,
    'max_depth'        : max_depth,
    'min_samples_split': min_samples_split,
    'max_features'     : max_features,
    'bootstrap'        : bootstrap
}


random_tuned_random_forest = RandomizedSearchCV(
    estimator = random_forest_clf_random,
    param_distributions = random_forest_random_cv,
    n_iter = 500,
    cv = 5,
    verbose=1,
    n_jobs=4)
random_tuned_random_forest.fit(X_train, y_train)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    9.9s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   47.5s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  3.2min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:  5.1min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:  7.3min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:  9.5min
[Parallel(n_jobs=4)]: Done 2500 out of 2500 | elapsed:  9.8min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=500, n_jobs=4,
          param_distributions={'criterion': ['gini', 'entropy'], 'n_estimators': [100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550...22, 232, 242], 'max_features': [5, 6, 7, 8, 'auto', 'sqrt', 'log2'], 'bootstrap': ['True', 'False']},
          pre_dispatch='2*n_jobs', ran

In [10]:
random_random_forest_score = random_tuned_random_forest.score(X_test,y_test)
print("score was")
print(random_random_forest_score)
random_random_forest_best_params = random_tuned_random_forest.best_params_
print("best parameters")
print(random_random_forest_best_params)

score was
0.7729257641921398
best parameters
{'n_estimators': 950, 'min_samples_split': 12, 'max_features': 'auto', 'max_depth': 10, 'criterion': 'entropy', 'bootstrap': 'False'}


In [11]:
#RandomForest with Grid HPT

random_forest_clf_grid = RandomForestClassifier() # Parameter Tuning here
#Parameters for Tuning
criterion         = ['gini']
n_estimators      = list(range(150,300,10))
max_depth         = list(range(10,20,1))
min_samples_split = list(range(2,25,2))
max_features      = [number_of_features] #list(range(5,10)) + ['auto','sqrt','log2']
bootstrap         = ['True','False']


random_forest_grid_cv = {
    'criterion'        : criterion,
    'n_estimators'     : n_estimators,
    'max_depth'        : max_depth,
    'min_samples_split': min_samples_split,
    'max_features'     : max_features,
    'bootstrap'        : bootstrap
}


grid_tuned_random_forest = GridSearchCV(
    estimator = random_forest_clf_grid,
    param_grid = random_forest_grid_cv,
    cv = 5,
    verbose=1,
    n_jobs =4)
grid_tuned_random_forest.fit(X_train, y_train)

Fitting 5 folds for each of 3600 candidates, totalling 18000 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    7.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   24.5s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   53.6s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:  3.4min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:  4.7min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed:  6.1min
[Parallel(n_jobs=4)]: Done 4042 tasks      | elapsed:  7.8min
[Parallel(n_jobs=4)]: Done 4992 tasks      | elapsed:  9.6min
[Parallel(n_jobs=4)]: Done 6042 tasks      | elapsed: 11.6min
[Parallel(n_jobs=4)]: Done 7192 tasks      | elapsed: 13.8min
[Parallel(n_jobs=4)]: Done 8442 tasks      | elapsed: 16.2min
[Parallel(n_jobs=4)]: Done 9792 tasks      | elapsed: 18.7min
[Parallel(n_jobs=4)]: Done 11242 tasks      | elapsed: 21.5min
[Parallel(n_jobs=4)]: Done 12792 tasks      | elapsed: 24.4min
[Parallel(

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'criterion': ['gini'], 'n_estimators': [150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290], 'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 'min_samples_split': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24], 'max_features': [9], 'bootstrap': ['True', 'False']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [12]:

best_estimator = grid_tuned_random_forest.best_estimator_ 

for i in [1,10,20]:
    estimator = best_estimator.estimators_[i]
    out_file_name = ("titanic_tree_best_random_forest_%i.dot" % i)
    out_png_file_name = ("titanic_tree_best_random_forest_%i.png" % i)
    export_graphviz(
        estimator,
        out_file=out_file_name,
        feature_names=feature_names,
        rounded=True,
        filled=True
    )
    #Converts dot file to PNG
    cmd_convert = 'dot -Tpng %s -o %s' % (out_file_name,out_png_file_name)
#    system(cmd_convert) # Why doesnt this work!
#    !dot -Tpng out_file_name -o out_png_file_name # Or this
    print(cmd_convert)  # Print to run manually as above doesnt work

dot -Tpng titanic_tree_best_random_forest_1.dot -o titanic_tree_best_random_forest_1.png
dot -Tpng titanic_tree_best_random_forest_10.dot -o titanic_tree_best_random_forest_10.png
dot -Tpng titanic_tree_best_random_forest_20.dot -o titanic_tree_best_random_forest_20.png


In [13]:
grid_random_forest_score = grid_tuned_random_forest.score(X_test,y_test)
print("score was")
print(grid_random_forest_score)
grid_random_forest_best_params = grid_tuned_random_forest.best_params_
print("best parameters")
print(grid_random_forest_best_params)

score was
0.8078602620087336
best parameters
{'bootstrap': 'True', 'criterion': 'gini', 'max_depth': 17, 'max_features': 9, 'min_samples_split': 12, 'n_estimators': 230}


In [14]:
print("Scoring Metrics for the different models are")

print("Score for Decision Tree with No Tuning     :%f" % dt_score)
print("Score for Decision Tree with Random Tuning :%f" % random_decision_tree_score)
print("Score for Decision Tree with Grid Tuning   :%f" % grid_decision_tree_score)
print("Score for Random Forest with No Tuning     :%f" % random_forest_score)
print("Score for Random Forest with Random Tuning :%f" % random_random_forest_score)
print("Score for Random Forest with Grid Tuning   :%f" % grid_random_forest_score)

Scoring Metrics for the different models are
Score for Decision Tree with No Tuning     :0.764192
Score for Decision Tree with Random Tuning :0.742358
Score for Decision Tree with Grid Tuning   :0.781659
Score for Random Forest with No Tuning     :0.772926
Score for Random Forest with Random Tuning :0.772926
Score for Random Forest with Grid Tuning   :0.807860


Scoring Metrics for the different models are
Score for Decision Tree with No Tuning     :0.764192
Score for Decision Tree with Random Tuning :0.742358
Score for Decision Tree with Grid Tuning   :0.781659
Score for Random Forest with No Tuning     :0.772926
Score for Random Forest with Random Tuning :0.772926
Score for Random Forest with Grid Tuning   :0.807860